In [1]:
import sagemaker
import boto3 
from sagemaker.image_uris import retrieve
from sagemaker.pytorch import PyTorch, PyTorchModel
import os 
from PIL import Image
from torchvision import transforms
import json
from sagemaker.amazon.amazon_estimator import get_image_uri
from datetime import datetime

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/dev/.config/sagemaker/config.yaml


In [4]:
sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()

region = sess.boto_session.region_name



In [3]:
bucket = os.getenv("BUCKET")
role_arn = os.getenv("ROLE_ARN")

In [5]:

if not bucket  and role_arn:
    
    raise ValueError("Las variables de entorno de SAGEMAKER no estan configuradas.")

else: print("Variables cargados exitosamente")

Variables cargados exitosamente


In [6]:
training_data = bucket + "data/"

# Train

### VGG16 & Resnet18

In [7]:
hyperparameters={
    'epochs': 10,                       # Aumentado para mejor convergencia
    'batch_size': 48,                   # Balance entre velocidad y estabilidad
    'learning_rate': 0.0003,            # Más bajo para mayor precisión
    'weight_decay': 0.00005,            # Regularización ligera
    'early_stopping': True,
    # 'class_weights': '"[0.15, 0.85]"',      # Ajuste fino para 300 defectuosas vs 2000 frescas
    'use_sampler': True,                # Crítico para detectar defectos
    'augmentation_intensity': 0.8,      # Máximo aumento para casos raros
    'dropout_rate': 0.4,                # Regularización moderada
    'patience': 10,                     # Más tolerancia para aprendizaje fino
    # 'optimizer': 'AdamW',               # Optimizador recomendado
    # 'scheduler': 'ReduceLROnPlateau',   # Ajuste automático de LR
    'val_split': 0.2,                   # Más datos para validación
    'threshold': 0.85                   # Umbral de confianza para producción
    }

In [8]:
# model_name = 'VGG16'
model_name = 'SimpleCNN'

In [9]:
estimator = PyTorch(
    entry_point='train.py',  
    source_dir='src',
    role=role_arn,
    instance_count=1,  
    instance_type='ml.g4dn.xlarge', 
    # instance_type ='ml.m5.large',
    framework_version='1.12.0', 
    py_version='py38',  
    hyperparameters=hyperparameters,
    base_job_name=f"{model_name}-{datetime.now().strftime('%Y%m%d-%H%M')}",
    output_path= bucket + "output",  
    sagemaker_session=sess,
    use_spot_intances=True,
    # max_wait=7200,
    max_run=3600,
)

In [10]:

estimator.fit(training_data)


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: SimpleCNN-20250519-1700-2025-05-19-22-00-59-316


2025-05-19 22:01:00 Starting - Starting the training job...
2025-05-19 22:01:24 Starting - Preparing the instances for training...
2025-05-19 22:01:52 Downloading - Downloading input data...
2025-05-19 22:02:37 Downloading - Downloading the training image........................
2025-05-19 22:06:27 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2025-05-19 22:06:39,695 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-05-19 22:06:39,730 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2025-05-19 22:06:39,762 sagemaker_pytorch_container.training INFO     Invoking user training script.
2025-05-19 22:06:40,091 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/op

### AWS MXnet 

In [8]:
hyperparameters = {
        "num_layers": 18,
        "image_shape": "3,224,224",
        "num_classes": 2,
        "num_training_samples": 2300,
        "mini_batch_size": 48,
        "epochs": 40,
        "learning_rate": 0.0003,
        "use_pretrained_model": 1
    }

In [ ]:
    
    # Obtener imagen del algoritmo
training_image = retrieve("image-classification", region=region, version="latest")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [ ]:


    
estimator = sagemaker.estimator.Estimator(
    training_image,
    role_arn,
    instance_count=1,
     instance_type='ml.g4dn.xlarge', 
    # instance_type = 'ml.p3.2xlarge,'
    # instance_type ='ml.m5.large',
    hyperparameters=hyperparameters
)
    

In [13]:
estimator.fit(training_data)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: image-classification-2025-05-19-17-47-00-458


2025-05-19 17:47:02 Starting - Starting the training job...
2025-05-19 17:47:18 Starting - Preparing the instances for training...
2025-05-19 17:47:45 Downloading - Downloading input data...
2025-05-19 17:48:36 Downloading - Downloading the training image......
2025-05-19 17:49:42 Training - Training image download completed. Training in progress....
2025-05-19 17:50:13 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
Nvidia gpu devices, drivers and cuda toolkit versions (only available on hosts with GPU):
Mon May 19 17:49:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr.

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 estimator.fit(training_data)                                                                 │
│   2                                                                                              │
│                                                                                                  │
│ /home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/telemetry/telem │
│ etry_logging.py:167 in wrapper                                                                   │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/telemetry/telem │
│ etry_logging.py:138 in wrapper                                                                   │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/workflow/pipeli │
│ ne_context.py:346 in wrapper                                                                     │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/si

# Metrics 

# Deploy


In [ ]:
model = PyTorchModel(
    model_data=estimator.model_data,
    role=role_arn,
    entry_point="inference.py",
    framework_version='1.13',
    py_version='py38',
    source_dir='code',
)

In [ ]:
# Desplegar el modelo
endpoint_name = 'dragon-fruit-endpoint'

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.xlarge', 
    endpoint_name=endpoint_name
)


# Inference

In [ ]:

sagemaker_runtime = boto3.client('sagemaker-runtime')

In [ ]:
image_path = 'Augmented_Dataset/normal/Fresh_Dragon_Augmented_Data0003.jpg'
image = Image.open(image_path)
image = image.resize((224, 224))  


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización estándar
])


In [ ]:
input_tensor = transform(image).unsqueeze(0) 
payload = json.dumps(input_tensor.numpy().tolist())

In [ ]:
# Hacer la predicción llamando al endpoint
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",  
)

In [ ]:
result = json.loads(response['Body'].read().decode())
print(f"Resultado de la predicción: {result}")

In [ ]:
# Función para visualizar imágenes con predicciones
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)


In [ ]:
# Visualización de algunas imágenes con predicciones
model.eval()
inputs, classes = next(iter(dataloaders['val']))
inputs = inputs.to(device)
classes = classes.to(device)

outputs = model(inputs)
_, preds = torch.max(outputs, 1)

plt.figure(figsize=(15, 15))
for i in range(min(batch_size, len(inputs))):
    ax = plt.subplot(min(batch_size, len(inputs)) // 8, 8, i + 1)
    ax.axis('off')
    ax.set_title(f'predicted: {class_names[preds[i]]}\ntrue: {class_names[classes[i]]}')
    imshow(inputs.cpu().data[i])

plt.show()


In [ ]:
predictor.delete_endpoint()